# Implementation of LSMC alghorithm 
#### Longstaff & Schwartz paper: https://people.math.ethz.ch/~hjfurrer/teaching/LongstaffSchwartzAmericanOptionsLeastSquareMonteCarlo.pdf 

In [1]:
import numpy as np
from scipy.stats import random_correlation, norm
import warnings
from sklearn.preprocessing import StandardScaler
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def pol_regression(degree, X_train, Y_train):
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import PolynomialFeatures
    "Creates a polynomial regression model for the given degree"
    poly_features = PolynomialFeatures(degree=degree, include_bias=True)

    # transforms the existing features to higher degree features.
    X_train = X_train
    X_train_poly = poly_features.fit_transform(X_train)

    # fit the transformed features to Linear Regression
    poly_model = LinearRegression(fit_intercept=False)
    poly_model.fit(X_train_poly, Y_train)

    # predicting on training data-set
    y_train_predicted = poly_model.predict(X_train_poly)
    
    return y_train_predicted

In [3]:
class LSMC:
    
    def __init__(self, op_type, S0, K, T, N, r, sigma, M):
        self.op_type = op_type
        self.S0 = S0
        self.K = K
        self.T = T
        self.N = N
        self.r = r
        self.sigma = sigma
        self.M = M
        self.dt = self.T / self.N
        self.df = np.exp(-self.r * self.dt)

    def price_paths(self):
        #np.random.seed(1)
        dW=np.random.standard_normal((self.M, self.N+1))*np.sqrt(self.dt)
        S = np.zeros((self.M, self.N+1))
        S[:,0] = self.S0
        for t in range(1, self.N + 1):
            S[:, t] = (S[:, t-1]*np.exp((self.r-0.5*self.sigma**2)*self.dt+self.sigma*dW[:, t-1]))
        return S
        
    def MCpayoff(self,stock_paths,K,option_type):
        if option_type == 'call':
            payoff = np.maximum(stock_paths - K,0)
        else:
            payoff = np.maximum(K - stock_paths,0)
        return payoff

    def price(self):
        self.S=self.price_paths()
        self.payoff = self.MCpayoff(self.S,self.K,self.op_type)
        value_matrix = np.zeros_like(self.payoff)
        value_matrix[:, -1] = self.payoff[:, -1]
        for t in range(self.N - 1, 0 , -1):
            x, y = self.S[:, t][self.payoff[:,t]>0], value_matrix[:, t+1][self.payoff[:,t]>0]*self.df
            continuation_value = np.zeros_like(self.payoff[:,t])
            continuation_value[self.payoff[:,t]>0] = pol_regression(2, x.reshape(-1,1),y)
            value_matrix[:, t] = np.where(self.payoff[:, t] > continuation_value,
                                          self.payoff[:, t],
                                          value_matrix[:, t+1] * self.df)

        return np.sum(value_matrix[:,1] * self.df) / self.M

In [4]:
op_type, S0, K, T, N, r, sigma, M = "put", 100.,100.,1.,50,0.01, 0.2, 100000

AmericanPUT = LSMC(op_type, S0, K, T, N, r, sigma, M)
AmericanPUT.price()

7.526565061140851